# Using BIP322 with a P2SH MultiSig Locking Script

## Signing Notebook

This notebook generates a BIP 0322 signature on the message "Hello World" using private key `L3VFeEujGtevx9w18HD1fhRbCH67Az2dpCymeRE1SoPK6XQtaN2k` which is listed as a test vector with a coresponding signature in the BIP.

A BIP 0322 signature involves producing two virtual Bitcoin transactions, which are unspendable on any network but otherwise valid. The first transaction, refered to as `to_spend`, encapsulates the message to be signed and the scriptPubKey which must be unlocked to produce a valid signature on this message. The second transaction, `to_sign`, is a transaction that takes the single tx_output of `to_spend` as an input and "spends" this output with a valid signature that unlocks the output and can be interpretted by any Bitcoin script engine. The resulting signature is stored in the witness of the `to_sign` transaction (unless a legacy version) and in the simple case it is this witness that is encoded and forms the signature that would be provided to a verifier.

* [BIP 0322](https://github.com/bitcoin/bips/blob/master/bip-0322.mediawiki)
* Implemented using the [buidl-python](https://github.com/buidl-bitcoin/buidl-python) library.

## Notebook Author

* Will Abramson
* [Legendary Requirements](http://legreq.com/)
* Contact: will@legreq.com

## Acknowledgements

This work was funded by Ryan Grant and [Digital Contract Design](https://contract.design/). Thanks also go to Joe Andrieu, Kalle Alm, Pieter Wuille and Jimmy Song for engaging with and supporting various aspects of this work.

## Notebook Steps

1. Generate a tagged hash of the message to sign (verify against test vector)
2. Import the provided private key and generate its associated address
3. Create the virtual to_spend transaction
4. Create unsigned virtual to_sign transaction
5. Sign the to_sign transaction using the private key (from 2.)

## 1. Generate a Tagged Hash of Message to Sign

In [8]:
from buidl.hash import tagged_hash

In [9]:
message = b"Hello World"

# The tag defined in BIP0322 that should be used
tag = b"BIP0322-signed-message"

tagged_hash = tagged_hash(tag,message)

### Verify Hash matches BIP 0322 Test Vector

https://github.com/bitcoin/bips/blob/master/bip-0322.mediawiki#message-hashing 

In [3]:
test_vector = 'f0eb03b1a75ac6d9847f55c624a99169b5dccba2a31f5b23bea77ba270de0a7a'
assert(test_vector == tagged_hash.hex())

## 2. Create 2-of-3 MultiSig Pay-to-Script Hash

Script commands are `2 <Public Key A> <Public Key B> <Public Key C> 3 CHECKMULTISIG`


In [2]:
from buidl.ecc import PrivateKey

secret_a = 287432
sk_a = PrivateKey(secret=secret_a)

secret_b = 187480
sk_b = PrivateKey(secret=secret_b)

secret_c = 286767
sk_c = PrivateKey(secret=secret_c)

In [37]:
from buidl.script import RedeemScript
commands = [2, sk_a.point.sec(), sk_b.point.sec(), sk_c.point.sec(), 3, 174]

redeem_script = RedeemScript(commands)

script_pubkey = script.script_pubkey()
address = script.address()

In [38]:
print(redeem_script)
print(script_pubkey)
print(address)

OP_[2] 0376aafb2f8a2503733df4d2d1eefde462d277ea1c9a437336b5d2c27de4894b1d 03e321dab12de7f0d9c11cc62a99536b06c11d239f539c538beb36b1a68be5fb24 0228c1122a8c869edc5cd204b999b8643689a5ad773ee1c788465c8e8957541128 OP_[3] OP_CHECKMULTISIG 
OP_HASH160 20fe1dfd52b356dc13ebf04470b8d44a56adb84e OP_EQUAL 
34hTy3vY3t3sND26koPHZtXv6tiohVq238


## 3. Create the virtual to_spend transaction

BIP 0322 defines the structure of this transaction as follows:

```python
    nVersion = 0
    nLockTime = 0
    vin[0].prevout.hash = 0000...000
    vin[0].prevout.n = 0xFFFFFFFF
    vin[0].nSequence = 0
    vin[0].scriptSig = OP_0 PUSH32[ message_hash ]
    vin[0].scriptWitness = []
    vout[0].nValue = 0
    vout[0].scriptPubKey = message_challenge
```

In [6]:
from buidl.tx import Tx, TxIn, TxOut
from buidl.script import Script,P2WPKHScriptPubKey
from buidl.helper import big_endian_to_int

### 3.1 Create the single transaction input

Note: the script commands `[0x00, 0x20, message_hash]`
- `0` is OP_0, which pushes a 0 onto the stack
- `32` is PUSH32 which pushes the next 32 bytes onto the stack
- `message_hash` is that next 32 bytes. Which is a tagged_hash of the message being signed.

**However, the implementation only requires commands [0, message_hash] be input. PUSH32 is implied by the size of the message_hash and added by the underlying library.**


In [10]:
# Not a valid Tx hash. Will never be spendable on any BTC network.
prevout_hash = bytes.fromhex('0000000000000000000000000000000000000000000000000000000000000000')
# prevout.n
prevout_index = big_endian_to_int(bytes.fromhex('FFFFFFFF'))
sequence = 0

# Byte array of message hash
message_hash = tagged_hash

# Note this used to be: commands = [0, 32, message_hash] (as per BIP)
# It appears the PUSH32 is implied and added by the size of the message added to the stack
commands = [0, message_hash]
scriptSig = Script(commands)
# Create Tx Input
tx_in = TxIn(prevout_hash,prevout_index,scriptSig,sequence)

In [11]:
tx_in

0000000000000000000000000000000000000000000000000000000000000000:4294967295

### 3.2 Create the single transaction output

In [12]:
# Value of tx output
value = 0

# Convert segwit address to script_pubkey
script_pubkey = script_pubkey

print(script_pubkey)
tx_out = TxOut(value,script_pubkey)

OP_HASH160 20fe1dfd52b356dc13ebf04470b8d44a56adb84e OP_EQUAL 


### 3.3 Create transaction using tx_in and tx_out

In [13]:
# create transaction
version=0
tx_inputs = [tx_in]
tx_outputs = [tx_out]
locktime=0
network="mainnet"

# Could be false, but using a segwit address. I think this is the "Simple Signature" in BIP-0322
segwit=True

virtual_to_spend_tx = Tx(version,tx_inputs,tx_outputs,locktime,network,segwit)

In [14]:
virtual_to_spend_tx


tx: 7320be304ad9309bcaca6a93b8ad4b6662a287f98ec560a9ab1175289fc2f2b0
version: 0
locktime: 0
tx_ins:
0000000000000000000000000000000000000000000000000000000000000000:4294967295
tx_outs:
0:OP_HASH160 20fe1dfd52b356dc13ebf04470b8d44a56adb84e OP_EQUAL 

## 4. Create virtual to_sign transaction

The structure as defined in BIP 0322:

```python
    nVersion = 0 or (FULL format only) as appropriate (e.g. 2, for time locks)
    nLockTime = 0 or (FULL format only) as appropriate (for time locks)
    vin[0].prevout.hash = to_spend.txid
    vin[0].prevout.n = 0
    vin[0].nSequence = 0 or (FULL format only) as appropriate (for time locks)
    vin[0].scriptWitness = message_signature
    vout[0].nValue = 0
    vout[0].scriptPubKey = OP_RETURN
```

### 4.1 Create single transaction input

In [15]:
from buidl.witness import Witness

In [16]:
# Identify the virtual_to_spend tx hash. 
pretx_hash = virtual_to_spend_tx.hash()
# Identifies the index of the output from the virtual_to_spend_tx to be "spent"
prevout_index = 0
sequence = 0


tx_input = TxIn(pretx_hash,prevout_index,script_sig=None,sequence=sequence)

### 4.2 Create single transaction output

In [17]:
value = 0
# OP Code 106 for OP_RETURN
commands = [106]
scriptPubKey = Script(commands)

tx_output = TxOut(value,scriptPubKey)
tx_output.serialize()
print(scriptPubKey)

OP_RETURN 


### 4.3 Create unsigned virtual to_sign transaction

In [18]:
# create transaction on mainnet
version=0
tx_inputs = [tx_input]
tx_outputs = [tx_output]
locktime=0
network="mainnet"

# Could be false, but using a segwit address. I think this is the "Simple Signature" in BIP-0322
segwit=True

virtual_to_sign_tx = Tx(version,tx_inputs,tx_outputs,locktime,network,segwit)

In [19]:
virtual_to_sign_tx.id()

'41074f45efd6f6311982173c7b9cd8816f3482689b224f0010576f303da01602'

## 5. Sign the virtual to_sign transaction

This populates the witness value on the only tx_input to the transaction

In [20]:
# Note: it is not currently set
print("Witness : ", virtual_to_sign_tx.tx_ins[0].witness)

Witness :  


### 5.1 Manually "fetch" the to_spend tx to populate scriptPubKey and Value in the to_sign tx input

Note: the prev_tx is the hash of the to_spend transaction, identifying it and cryptographically binding it to the to_sign transaction. Normally, a Bitcoin node would query the network for the transaction identified by this hash. In BIP 0322 this must be manually done.

In [23]:
# Have to manually set these, because cannot FETCH to_spend from network
# Note: Would ideally submit a P.R. to the buidl library to support this
virtual_to_sign_tx.tx_ins[0]._script_pubkey = script_pubkey
virtual_to_sign_tx.tx_ins[0]._value = 0


### 5.2 Sign the only tx_input

Note: the tx_input is identified as the output from the to_spend transaction

In [30]:
script.is_p2sh_multisig()

True

In [32]:
from buidl.psbt import PSBT

In [39]:
redeem_lookup = {}
redeem_lookup[redeem_script.commands[1]] = redeem_script 

In [40]:
psbt = PSBT.create(virtual_to_sign_tx, redeem_lookup=redeem_lookup)

In [42]:
psbt.sign_with_private_keys([sk_a])

False

In [27]:
# print(virtual_to_sign_tx.tx_ins[0]._script_pubkey)
virtual_to_sign_tx.sign_input(0,sk_a, redeem_script=script)



RuntimeError: Unknown ScriptPubKey

In [26]:
virtual_to_sign_tx.tx_ins[0]._script_pubkey

OP_HASH160 20fe1dfd52b356dc13ebf04470b8d44a56adb84e OP_EQUAL 

In [23]:
# Now the witness has been populated
print("Witness : ", virtual_to_sign_tx.tx_ins[0].witness)

Witness :  3044022004a59ea55271540152a988f07c9f35098b81ee00e5e7913c709bc28c7e8ba48e0220620246b02e5da7ff849b172923b9b2862deb15fb06dfad2bda163e28350f201501 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872 


In [24]:
virtual_to_sign_tx.verify_input(0)

True

## 6. Serialize the Witness and Base64 Encode

From the BIP-0322:

`A simple signature consists of a witness stack, consensus encoded as a vector of vectors of bytes, and base64-encoded. Validators should construct to_spend and to_sign as defined below, with default values for all fields except that`

In [25]:
from buidl.helper import base64_encode
# 
bip322_signature = base64_encode(virtual_to_sign_tx.serialize_witness())
print("Copy across this signature output and use it in the verification notebook: \n")
print(bip322_signature)

Copy across this signature output and use it in the verification notebook: 

AkcwRAIgBKWepVJxVAFSqYjwfJ81CYuB7gDl55E8cJvCjH6LpI4CIGICRrAuXaf/hJsXKSO5soYt6xX7Bt+tK9oWPig1DyAVASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI=


## 7. Verify Signature against BIP 0322 Test Vector

Test vector taken from BIP0322

### NOTE: This currently does not pass. Submitted a P.R to update the BIP 0322 signature test vector as it is currently incorrect

In [26]:
sig_test_vector = 'AkcwRAIgG3PASL/vRTgAqogWT6S8rUOQXNnfRzX6JncmbFlHc1ACIGQdsW+rnVmsQzyAYRQisHKFMigDmKiL7LUw4x17Fw5tASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI='
assert(sig_test_vector == bip322_signature)

AssertionError: 

In [ ]:
btc_core_sig_test_vector = 'AkcwRAIgZRfIY3p7/DoVTty6YZbWS71bc5Vct9p9Fia83eRmw2QCICK/ENGfwLtptFluMGs2KsqoNSk89pO7F29zJLUx9a/sASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI='
assert(btc_core_sig_test_vectore == bip322_signature)